This notebook continuously estimates the prevailing discrete/realtime positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>
<br>

## Preliminaries: In Progress

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.explore.prospects
import atlantic.explore.hospitalisations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

### Attributes

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 'hospitalizedIncrease': np.float64,          
         'positiveCumulative': np.float64, 'testCumulative': np.float64, 'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 
         'positiveRate': np.float64, 'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>

### Source URL

In [14]:
datauri = os.path.join(parent, 'warehouse', 'baselines.csv')

<br>

### Data

In [15]:
baselines = pd.read_csv(filepath_or_buffer=datauri, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13312 entries, 0 to 13311
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          13312 non-null  datetime64[ns]
 1   STUSPS                  13312 non-null  object        
 2   deathIncrease           13312 non-null  float64       
 3   deathCumulative         13312 non-null  float64       
 4   positiveIncrease        13312 non-null  float64       
 5   positiveCumulative      13312 non-null  float64       
 6   hospitalizedIncrease    13312 non-null  float64       
 7   hospitalizedCumulative  13312 non-null  float64       
 8   testIncrease            13312 non-null  float64       
 9   testCumulative          13312 non-null  float64       
 10  deathRate               13312 non-null  float64       
 11  positiveRate            13312 non-null  float64       
 12  testRate                13312 non-null  float6

<br>
<br>

## Regression

### $\small{Set-up}$

#### Latest Date

In [16]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2020-10-03 00:00:00


<br>
<br>

#### Periods

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [17]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

#### Partitions

In [18]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### $positives$ $\quad vs.\ \;$ $tests$

#### Analysis

In [19]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [20]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

Estimates

In [21]:
logger.info(estimates.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetimeobject           52 non-null     datetime64[ns]
 1   gradient                 52 non-null     object        
 2   lowerconfidenceinterval  52 non-null     object        
 3   upperconfidenceinterval  52 non-null     object        
 4   pvalue                   52 non-null     object        
 5   rsquared                 52 non-null     object        
 6   intercept                52 non-null     object        
 7   STUSPS                   52 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.4+ KB


<br>

Predictions

In [22]:
logger.info(predictions.head())

INFO:__main__:   testRate      minimum       median      maximum STUSPS
0   24000.0  3229.512045  3232.177949  3242.672801     AL
1   24100.0  3241.744757  3244.728421  3256.085402     AL
2   24200.0  3253.977470  3257.278892  3269.498003     AL
3   24300.0  3266.210183  3269.829364  3282.910604     AL
4   24400.0  3278.442896  3282.379836  3296.323205     AL


<br>

#### State of affairs

In [23]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [24]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [25]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
31,2020-10-03,0.0292768,0.0244004,0.0385281,8.86872e-17,0.987934,114.312,NM,2.92768,3.207181,48.0,34.1568
29,2020-10-03,0.0135565,0.00588841,0.043202,7.13679e-09,0.883299,340.438,NH,1.35565,3.075146,49.0,73.7652
1,2020-10-03,0.0291654,0.020108,0.0351245,4.04101e-15,0.98056,-628.309,AK,2.91654,1.941233,50.0,34.2872
19,2020-10-03,0.00498465,0.00392967,0.00687165,4.61325e-15,0.980236,239.538,ME,0.498465,1.220989,51.0,200.616
45,2020-10-03,0.00577894,0.00466781,0.00867189,5.46689e-14,0.97492,130.343,VT,0.577894,1.072467,52.0,173.042


<br>
<br>

### $hospitalisations$ $\; vs.\ \;$ $positives$

In [26]:
del formula, regressor, prospects, estimates, predictions

<br>

#### Data

In [27]:
hospitalisations = atlantic.explore.hospitalisations.Hospitalisations(blob=baselines, warehouse=os.path.join(parent, 'warehouse'))

In [28]:
hospital = hospitalisations.exc(limit=periods.min())

INFO:atlantic.explore.hospitalisations:
Hospitalized Positives Curves:
None



<class 'pandas.core.frame.DataFrame'>
Int64Index: 9216 entries, 0 to 9215
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            9216 non-null   datetime64[ns]
 1   STUSPS                    9216 non-null   object        
 2   hospitalizedRate          9216 non-null   float64       
 3   positiveRate              9216 non-null   float64       
 4   ndays                     9216 non-null   int64         
 5   hospitalizedPositiveRate  9216 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 504.0+ KB


<br>

#### Analysis

In [29]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [30]:
estimates, predictions = prospects.exc(partitions=hospital[partitionby].unique())

<br>

#### State of Affairs

In [31]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [32]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [33]:
hpr = hpr.merge(hospital[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
31,2020-10-03,0.0287533,0.0275462,0.0338801,5.68732e-21,0.997224,61.7252,WI,2.87533,32.0,5.498471,34.7786
32,2020-10-03,0.0278462,0.0234362,0.0362654,6.28053e-15,0.98244,55.0875,NE,2.78462,33.0,5.064180,35.9115
33,2020-10-03,0.0268751,0.0140549,0.0339822,1.26145e-10,0.948919,37.8269,NH,2.68751,34.0,8.642550,37.2091
34,2020-10-03,0.0260411,0.0202879,0.0313761,1.15708e-15,0.985129,20.1032,WY,2.60411,35.0,4.414768,38.4008
35,2020-10-03,0,-3.77302e-13,3.77302e-13,0.958777,-2277.45,462.615,NY,0,36.0,19.422263,0


<br>
<br>

## Clean-up

In [34]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states